[![Binder](https://mybinder.org/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia-14/main?filepath=02%20Tips%20and%20Tricks%20and%20DisjunctiveProgramming.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>

# Tips and Tricks and DisjunctiveProgramming.jl

Expressing standard things in JuMP.

If you are new to JuMP you will be surprised how restrictive mathematical optimization is.

In [ ]:
using JuMP, HiGHS

## Tips and Tricks

Luckily JuMP comes with a good overview about common pitfalls. Definitely checkout the following Tips and Tricks sections.

- [Linear programs T&T](https://jump.dev/JuMP.jl/stable/tutorials/linear/tips_and_tricks/)
- [Nonlinear programs T&T](https://jump.dev/JuMP.jl/stable/tutorials/nonlinear/tips_and_tricks/)
- [Conic programs T&T](https://jump.dev/JuMP.jl/stable/tutorials/conic/tips_and_tricks/)

### max, min

The `max` Function is nonlinear but can be partly expressed as a linear program.

`t ≥ max{x,y}`

In [ ]:
model_max = Model(HiGHS.Optimizer);
@variable(model_max, t)
@variable(model_max, x)
@variable(model_max, y)
@constraint(model_max, t >= x)
@constraint(model_max, t >= y)
@objective(model_max, Min, t)

fix(x, 4)
fix(y, 5)

set_attribute(model_max, "output_flag", false)  # hide HiGHS optimize! output 
optimize!(model_max)
@assert is_solved_and_feasible(model_max)
value(t)

**👉 Your Challenge:** Change `x` and `y` to see that it actually works.

`t ≥ min{x,y}`

This formulation is not possible in linear programs. For nonlinear programs you can use [nested optimizations](https://jump.dev/JuMP.jl/stable/tutorials/nonlinear/nested_problems/).

### other T&T


**👉 Your Challenge:** Take a look at another T&T for linear programs.

In [ ]:
# Your Space
# ...

The following linear T&T are better done with `DisjunctiveProgramming`:
- Boolean operators
- Disjunctions
- Indicator Constraints

## Disjunctive Programming

remember our initial optimization example?

```julia
@variable(model, v >= 0)
@variable(model, b, Bin)

@constraint(model, v >= 4*b)  # if b == 1, this constraint is "active"
@constraint(model, v >= 6*(1-b))  # if b == 0, this constraint is "active"

@objective(model, Min, v)
```

This is a kind of disjunction, an OR relation among constraints.

In [ ]:
using DisjunctiveProgramming
model_or = GDPModel(HiGHS.Optimizer)  # More expressive version of Model

### Logical Variables (instead of Binary)

DisjunctiveProgramming works best when using one logical variable per alternative.

In [ ]:
@variable(model_or, Y[1:2], Logical)

These can be restricted such e.g. that only one holds true.

In [ ]:
@constraint(model_or, Y in Exactly(1))  # logical constraint set

### Disjunctive Constraints

In order to connect a constraint to a given alternative, add an extra argument like `Disjunct(Y[1])`.

**👉 Your Challenge:** What kind of OR relation is defined here?

In [ ]:
@variable(model_or, x[1:2])

@constraint(model_or, x[1] + 1 == x[2], Disjunct(Y[1]))
@constraint(model_or, x[2] + 1 == x[1], Disjunct(Y[2]))

@objective(model_or, Min, sum(x))

Importantly, these constraints only become active if the logical variables are further specified in `@disjunction`

In [ ]:
@disjunction(model_or, Y, exactly1=false)

By default `@disjunction` does two things
- Informing the model which Disjunctive Constraints to expand (you get wrong results without it)
- adding Exactly1 constraints

Separating both is more generic and will help us below.

In [ ]:
# when fixing another value, you need to reexecute all cells
fix(x[1], 3)

In [ ]:
set_attribute(model_or, "output_flag", false)  # hide HiGHS optimize! output 
optimize!(model_or)
round.(value.(x))

## Scheduling Tasks to Workers

Let's look at a common real-world example.

In [ ]:
# Symbols are needed because of https://github.com/hdavid16/DisjunctiveProgramming.jl/issues/112
tasks = Symbol.([
    "repair bicycle 1 front weel"
    "repair bicycle 2 light"
    "order 4 weels"
    "be at the service desk"
    "exchange old bikes with new bikes"
    "call customer Mr. Yu"
    "relax"
])

In [ ]:
workers = Symbol.([
    "Nora"
    "Omar"
])

In [ ]:
model_bicycle = GDPModel(HiGHS.Optimizer)
# for each task we create a time variable
@variable(model_bicycle, 0 <= times[tasks] <= 200);

Use complex indices to represent our assignment from task to worker as logical variables.

In [ ]:
@variable(model_bicycle, assign[tasks, workers], Logical)

✅ Each task should be assigned to one worker.

In [ ]:
@constraint(model_bicycle, [t=tasks], assign[t, :] in Exactly(1));  # a logical constraint set

✅ All tasks per worker should not overlap.

We represent this as "pairs which belong to the same worker do not overlap"

In [ ]:
task_pairs = [(tasks[i], tasks[j]) for i in 1:length(tasks)-1 for j in i+1:length(tasks)]

In [ ]:
@variable(model_bicycle, assign_pair[task_pairs, workers], Logical)

# restrict assign_pair to follow assign
@constraint(model_bicycle, [p=task_pairs, w=workers],
    # parentheses are CRUCIAL!
    assign_pair[p, w] == (assign[p[1], w] && assign[p[2], w]) := true)

# now we can create disjunctive constraints
@constraint(model_bicycle, [p=task_pairs, w=workers], 
    times[p[1]] + 1 <= times[p[2]], Disjunct(assign_pair[p, w]));

In [ ]:
# again, all logical variables used inside Disjunct tags need to passed to @disjunction
@disjunction(model_bicycle, assign_pair, exactly1=false);

### Solve it

In [ ]:
@objective(model_bicycle, Min, sum(times))
set_attribute(model_bicycle, "output_flag", false)  # hide HiGHS optimize! output 
optimize!(model_bicycle)
@assert is_solved_and_feasible(model_bicycle)

In [ ]:
round.(value.(times))

In [ ]:
round.(Int, value.(binary_variable.(assign)))

**👉 Your Challenge:** Add further tasks / further workers.

# Thank you

For questions or suggestions please contact me at stephan.sahm@jolin.io

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>